In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

In [2]:
df_admet_prediction = pd.DataFrame()

HLM

In [3]:
data = pd.read_csv('featurized_data_admet.csv').drop(columns=['my_id', 'LogD', 'MDR1-MDCKII', 'KSOL', 'MLM'])
data

,SMILES,HLM,training_set,0,1,2,3,4,5,6,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,NaN,True,1.171046,-1.709964,-1.232664,1.860159,-1.148111,-1.168615,1.584990,...,0.144337,1.531990,-0.810303,0.296689,-0.621365,-1.106144,0.762330,-0.761378,-1.274996,-0.460258
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,NaN,True,-2.745084,-1.049907,1.334981,1.905276,1.214691,0.937328,-2.540276,...,0.544986,1.725864,-0.310606,0.979101,0.459233,0.538236,-0.252584,-0.212149,-0.018674,-0.152451
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,NaN,True,-0.976000,-1.049323,3.051156,2.945424,2.015631,-1.640104,-0.486444,...,0.822131,1.796755,0.109316,1.148936,0.219072,0.434542,0.089569,0.211311,-0.209988,0.574256
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,NaN,True,-2.336900,-1.334151,-0.813121,5.999913,3.993001,-2.829812,-1.460422,...,0.330147,0.818578,-0.157749,0.460316,0.029226,1.010529,0.828361,0.412284,-1.456596,0.550043
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,NaN,True,-2.976158,0.377570,0.348468,2.663868,3.342389,-1.316121,-1.861355,...,0.974805,0.831901,-0.274851,1.695331,-0.544704,-0.302338,0.599298,-0.803252,-0.909561,0.617987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.600852,1.758608,3.050236,-2.634969,2.354528,-1.809476,-6.122465,...,0.676559,1.834212,-0.208483,1.160509,-0.379795,1.017874,0.023464,0.889000,-0.436904,1.477911
556,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.212416,1.016423,3.263524,-2.550871,2.011647,-2.047450,-6.484405,...,0.799475,1.806010,-0.320671,1.235600,-0.248747,1.048957,0.051964,0.893613,-0.487688,1.360138
557,COC[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN...,NaN,False,-0.356645,2.175031,3.133271,-2.253229,1.212674,-1.492542,-6.595284,...,1.134350,1.222514,-0.517434,1.147382,-0.472114,-0.133162,0.201345,0.679035,-0.680492,0.537756
558,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.364247,2.796600,3.689862,-2.036362,1.520496,-1.304389,-7.439854,...,0.973330,1.113834,-0.604300,0.977338,-0.599096,0.034397,0.263963,0.769886,-1.035524,1.216965


In [4]:
train_data = data[(data.training_set) & (~data['HLM'].isna())].reset_index(drop=True).drop(columns=['training_set'])
test_data = data[~data.training_set].reset_index(drop=True).drop(columns=['training_set']).drop(columns='HLM')

In [5]:
train_data

,SMILES,HLM,0,1,2,3,4,5,6,7,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,CN(C1=CC=C2CNCC2=C1)[C@H](C(=O)NCC(F)F)C1=CC(C...,17.1,-3.639486,-0.198837,1.908512,1.170709,1.201834,0.847216,-4.171668,0.785983,...,0.360414,1.312590,-0.130845,0.348302,0.605345,0.847359,-0.282226,-0.010332,-0.150074,-0.008987
1,N#CC1=CC2=C(C=C1NC(=O)C1=CC(F)=CC3=C1C=NN3)CNC2,16.7,0.264225,-4.649776,-2.142826,2.155019,-1.289509,-2.665573,1.054727,0.852086,...,0.398534,0.946872,0.087902,1.324152,0.179993,1.181708,-0.081205,0.359662,-0.452750,0.876915
2,O=C(NC1=CC=C2CNCC2=C1)C1=CC(F)=CC(C2=CNN=C2)=C1,36.3,-0.766942,-2.551304,0.042937,1.717186,0.690480,-2.250106,0.341561,3.575093,...,0.332091,0.356447,0.053039,1.404808,0.984646,1.858472,0.417110,0.204112,-1.085464,1.243966
3,COCCNC(=O)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Cl)=CC...,19.7,-2.442127,-0.750176,2.262574,1.851896,1.943286,0.685758,-3.423627,0.817352,...,0.280456,0.996561,-0.167144,0.939608,0.720415,0.120833,0.049576,-0.532240,-0.837514,-0.254165
4,O=C(NC1=CC(Br)=C2CNCC2=C1)C1=CC(F)=CC2=C1C=NN2,13.6,-0.824766,-3.934841,0.128877,4.133136,0.060160,-1.451824,-0.055768,0.557559,...,0.338323,0.790335,-0.140245,0.888108,0.487913,1.214096,0.413354,0.288804,-0.793192,0.675239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,C=CC(=O)N1CCCC2=CC=C(N(CC3=CC=CC(C(F)(F)F)=C3)...,614.0,-1.693605,-0.514387,2.921044,-1.084245,2.741452,-2.702383,-1.858153,0.296535,...,1.270796,2.009364,0.403548,0.753315,0.527727,1.155271,0.560215,1.148774,-0.139170,0.997934
297,C=CC(=O)N1CCCC2=CC=C(N(CC3=CC=CC(Cl)=C3)C(=O)C...,752.0,-0.568220,0.108331,3.653222,-0.142993,1.791349,-3.624483,-3.076594,0.537416,...,0.900786,1.682289,0.231539,0.715634,0.235459,0.634066,0.183143,1.351036,-0.065795,0.444673
298,C=CC(=O)N1CCCC2=CC=C(N(C(=O)CC3=CN=CC4=CC=CC=C...,1380.0,-1.769146,1.957387,0.738775,-0.329697,2.399386,-4.285360,-1.878670,0.033536,...,1.390941,1.621874,0.179236,0.187389,0.436384,-0.077299,0.306170,0.950794,-0.867420,0.845003
299,CC(C)NC[C@H](O)COC1=CC=CC2=CC=CC=C12 |&1:5|,25.5,-1.399064,-1.804412,-4.436469,2.023061,1.168960,-0.661248,5.615030,-4.620008,...,1.173786,2.097678,-0.498668,0.799745,1.768006,2.232763,0.311292,0.462726,-2.909488,0.410403


In [6]:
test_data

,SMILES,0,1,2,3,4,5,6,7,8,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,CC(C)[C@H]1C2=C(CCN1C(=O)CC1=CN=CC3=CC=CC=C13)...,-1.960082,3.442454,0.068059,-3.082606,-0.711921,-2.256452,-3.024943,-0.400217,0.186137,...,1.500217,1.362507,-0.593836,1.102913,-0.202193,1.493693,0.971354,1.292989,-1.575871,0.794808
1,CC(C)C1(C)CCN(C(=O)CC2=CN=CC3=CC=CC=C23)CC1,1.888740,1.872707,0.580080,1.415552,3.370449,-1.690022,0.570753,-0.657132,-1.073991,...,1.312542,1.701150,0.004610,1.613848,0.335976,0.710924,0.902466,1.648935,-1.617046,1.269374
2,O=C(CC1=CN=CC2=CC=CC=C12)N1CCC2=C(C=CS2)C12CCC2,-1.014984,3.478197,0.670941,-2.388297,-0.010947,-2.244923,-3.819330,-1.084298,-0.141481,...,1.129475,1.257946,-0.256014,0.959213,0.059674,1.096273,0.341441,1.063027,-0.676640,0.739956
3,CC1(CC(F)(F)F)CCN(C(=O)CC2=CN=CC3=CC=CC=C23)CC1,1.228055,0.678481,0.425739,1.863930,2.261316,-1.173955,1.096332,-0.240087,0.200905,...,0.998078,2.490732,0.200787,2.023941,0.530019,1.123201,0.511525,1.135225,-0.128001,0.617469
4,CCC1=CC=C([C@H]2C[C@H](C)CCN2C(=O)CC2=CN=CC3=C...,0.272685,-1.378255,0.473770,-1.439623,1.638330,-2.546845,-1.649766,-2.129847,1.081641,...,0.888903,1.657008,-0.136771,1.139195,0.289371,1.491972,1.229188,1.485757,-1.206364,1.061064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,0.600852,1.758608,3.050236,-2.634969,2.354528,-1.809476,-6.122465,2.140939,-5.439906,...,0.676559,1.834212,-0.208483,1.160509,-0.379795,1.017874,0.023464,0.889000,-0.436904,1.477911
122,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,0.212416,1.016423,3.263524,-2.550871,2.011647,-2.047450,-6.484405,2.759993,-5.453090,...,0.799475,1.806010,-0.320671,1.235600,-0.248747,1.048957,0.051964,0.893613,-0.487688,1.360138
123,COC[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN...,-0.356645,2.175031,3.133271,-2.253229,1.212674,-1.492542,-6.595284,2.060108,-4.649041,...,1.134350,1.222514,-0.517434,1.147382,-0.472114,-0.133162,0.201345,0.679035,-0.680492,0.537756
124,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,0.364247,2.796600,3.689862,-2.036362,1.520496,-1.304389,-7.439854,2.283727,-5.042495,...,0.973330,1.113834,-0.604300,0.977338,-0.599096,0.034397,0.263963,0.769886,-1.035524,1.216965


In [7]:
task = 'HLM'
kernel = 'poly'
C = 2.8
epsilon = 0.0001
seed = 1

train_data = train_data.set_index('SMILES', drop=True)
train_data = shuffle(train_data, random_state=seed)
X_train = train_data.drop(columns=task)
y_train = train_data[task]

test_data = test_data.set_index('SMILES', drop=True)
X_test = test_data

current_model = SVR(kernel=kernel, C=C, epsilon=epsilon)
current = make_pipeline(MinMaxScaler(), current_model)
current.fit(X_train,y_train)
y_pred = current.predict(X_test)

In [8]:
y_pred

array([285.04236394, 346.42294141, 227.31611498, 248.09506748,
       186.98596097, 183.2831017 , 175.889201  , 302.24277778,
       277.3018058 ,  69.08767533,  72.48850035, 196.48243629,
        88.83430318, 322.29150921,  78.60967858, 337.97375548,
       339.01135273, 330.90122105, 103.7085081 , 358.8231401 ,
        99.43925353, 235.10085712, 231.96500418, 356.42805704,
       483.67767706, 410.15084875, 245.43488864, 289.00453677,
       282.6155715 , 488.2001032 , 237.04312227, 369.73866284,
       214.38900733, 245.43488864, 270.47589109, 281.32493722,
       290.12323131, 282.6844503 , 280.63654469, 290.60296814,
       289.05657993, 296.41519102, 318.26576368, 174.55253511,
       172.24219146, 261.60903127, 257.31001747, 257.31001747,
       274.30061692, 532.59584942, 407.68775989, 275.04219712,
       338.19905993, 326.5047657 , 278.38057482, 322.98772919,
       111.10287245, 272.6785879 , 295.52703064, 265.10234347,
       417.15460871, 360.40388189, 361.52016352, 276.40

In [9]:
df_admet_prediction['HLM'] = y_pred
df_admet_prediction

,HLM
0,285.042364
1,346.422941
2,227.316115
3,248.095067
4,186.985961
...,...
121,103.581036
122,103.028499
123,75.458646
124,152.454394


KSOL

In [10]:
data = pd.read_csv('featurized_data_admet.csv').drop(columns=['my_id', 'LogD', 'MDR1-MDCKII', 'HLM', 'MLM'])
data

,SMILES,KSOL,training_set,0,1,2,3,4,5,6,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,NaN,True,1.171046,-1.709964,-1.232664,1.860159,-1.148111,-1.168615,1.584990,...,0.144337,1.531990,-0.810303,0.296689,-0.621365,-1.106144,0.762330,-0.761378,-1.274996,-0.460258
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,333.0,True,-2.745084,-1.049907,1.334981,1.905276,1.214691,0.937328,-2.540276,...,0.544986,1.725864,-0.310606,0.979101,0.459233,0.538236,-0.252584,-0.212149,-0.018674,-0.152451
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,NaN,True,-0.976000,-1.049323,3.051156,2.945424,2.015631,-1.640104,-0.486444,...,0.822131,1.796755,0.109316,1.148936,0.219072,0.434542,0.089569,0.211311,-0.209988,0.574256
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,376.0,True,-2.336900,-1.334151,-0.813121,5.999913,3.993001,-2.829812,-1.460422,...,0.330147,0.818578,-0.157749,0.460316,0.029226,1.010529,0.828361,0.412284,-1.456596,0.550043
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,375.0,True,-2.976158,0.377570,0.348468,2.663868,3.342389,-1.316121,-1.861355,...,0.974805,0.831901,-0.274851,1.695331,-0.544704,-0.302338,0.599298,-0.803252,-0.909561,0.617987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.600852,1.758608,3.050236,-2.634969,2.354528,-1.809476,-6.122465,...,0.676559,1.834212,-0.208483,1.160509,-0.379795,1.017874,0.023464,0.889000,-0.436904,1.477911
556,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.212416,1.016423,3.263524,-2.550871,2.011647,-2.047450,-6.484405,...,0.799475,1.806010,-0.320671,1.235600,-0.248747,1.048957,0.051964,0.893613,-0.487688,1.360138
557,COC[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN...,NaN,False,-0.356645,2.175031,3.133271,-2.253229,1.212674,-1.492542,-6.595284,...,1.134350,1.222514,-0.517434,1.147382,-0.472114,-0.133162,0.201345,0.679035,-0.680492,0.537756
558,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.364247,2.796600,3.689862,-2.036362,1.520496,-1.304389,-7.439854,...,0.973330,1.113834,-0.604300,0.977338,-0.599096,0.034397,0.263963,0.769886,-1.035524,1.216965


In [11]:
train_data = data[(data.training_set) & (~data['KSOL'].isna())].reset_index(drop=True).drop(columns=['training_set'])
test_data = data[~data.training_set].reset_index(drop=True).drop(columns=['training_set']).drop(columns='KSOL')

In [12]:
task = 'KSOL'
kernel = 'poly'
C = 9
epsilon = 0.5
seed = 1

train_data = train_data.set_index('SMILES', drop=True)
train_data = shuffle(train_data, random_state=seed)
X_train = train_data.drop(columns=task)
y_train = train_data[task]

test_data = test_data.set_index('SMILES', drop=True)
X_test = test_data

current_model = SVR(kernel=kernel, C=C, epsilon=epsilon)
current = make_pipeline(MinMaxScaler(), current_model)
current.fit(X_train,y_train)
y_pred = current.predict(X_test)

In [13]:
y_pred

array([235.35186906, 283.14861615, 240.03864644, 248.13327953,
       376.53225331, 341.09000387, 285.91281328, 280.81565722,
       374.35414365, 327.15644519, 378.24640597, 260.90985913,
       356.95379042, 357.63791339, 334.51902209, 340.21320105,
       319.40600118, 341.3447968 , 349.50684668, 274.20248015,
       398.32203443, 326.50944201, 293.96864556, 303.05092418,
        96.38585934,  77.52721625, 256.3267322 , 442.36608405,
       396.58051879,  73.72623113, 329.43076094, 208.57587102,
       327.85975054, 256.3267322 , 403.13518364, 325.30508792,
       428.13669809, 367.71367128, 306.72259312, 342.09278798,
       343.85040587, 366.84358413, 342.56854889, 402.03376623,
       379.56561395, 424.43407157, 264.98677406, 264.98677406,
       293.58083598, 107.03292808,  66.87233236, 257.23559894,
       375.32798657, 372.93907044, 364.17704955, 359.76004417,
       378.49952748, 343.17072062, 405.65036048, 289.08480006,
        98.02854164, 376.43817446, 397.74826047, 269.78

In [14]:
df_admet_prediction['KSOL'] = y_pred
df_admet_prediction

,HLM,KSOL
0,285.042364,235.351869
1,346.422941,283.148616
2,227.316115,240.038646
3,248.095067,248.133280
4,186.985961,376.532253
...,...,...
121,103.581036,381.535459
122,103.028499,371.569023
123,75.458646,378.258895
124,152.454394,424.290765


LogD

In [15]:
data = pd.read_csv('featurized_data_admet.csv').drop(columns=['my_id', 'KSOL', 'MDR1-MDCKII', 'HLM', 'MLM'])
data

,SMILES,LogD,training_set,0,1,2,3,4,5,6,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,0.3,True,1.171046,-1.709964,-1.232664,1.860159,-1.148111,-1.168615,1.584990,...,0.144337,1.531990,-0.810303,0.296689,-0.621365,-1.106144,0.762330,-0.761378,-1.274996,-0.460258
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,2.9,True,-2.745084,-1.049907,1.334981,1.905276,1.214691,0.937328,-2.540276,...,0.544986,1.725864,-0.310606,0.979101,0.459233,0.538236,-0.252584,-0.212149,-0.018674,-0.152451
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,0.4,True,-0.976000,-1.049323,3.051156,2.945424,2.015631,-1.640104,-0.486444,...,0.822131,1.796755,0.109316,1.148936,0.219072,0.434542,0.089569,0.211311,-0.209988,0.574256
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,1.0,True,-2.336900,-1.334151,-0.813121,5.999913,3.993001,-2.829812,-1.460422,...,0.330147,0.818578,-0.157749,0.460316,0.029226,1.010529,0.828361,0.412284,-1.456596,0.550043
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,-0.3,True,-2.976158,0.377570,0.348468,2.663868,3.342389,-1.316121,-1.861355,...,0.974805,0.831901,-0.274851,1.695331,-0.544704,-0.302338,0.599298,-0.803252,-0.909561,0.617987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.600852,1.758608,3.050236,-2.634969,2.354528,-1.809476,-6.122465,...,0.676559,1.834212,-0.208483,1.160509,-0.379795,1.017874,0.023464,0.889000,-0.436904,1.477911
556,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.212416,1.016423,3.263524,-2.550871,2.011647,-2.047450,-6.484405,...,0.799475,1.806010,-0.320671,1.235600,-0.248747,1.048957,0.051964,0.893613,-0.487688,1.360138
557,COC[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN...,NaN,False,-0.356645,2.175031,3.133271,-2.253229,1.212674,-1.492542,-6.595284,...,1.134350,1.222514,-0.517434,1.147382,-0.472114,-0.133162,0.201345,0.679035,-0.680492,0.537756
558,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.364247,2.796600,3.689862,-2.036362,1.520496,-1.304389,-7.439854,...,0.973330,1.113834,-0.604300,0.977338,-0.599096,0.034397,0.263963,0.769886,-1.035524,1.216965


In [16]:
train_data = data[(data.training_set) & (~data['LogD'].isna())].reset_index(drop=True).drop(columns=['training_set'])
test_data = data[~data.training_set].reset_index(drop=True).drop(columns=['training_set']).drop(columns='LogD')

In [17]:
task = 'LogD'
kernel = 'poly'
C = 0.035
epsilon = 0.05
seed = 1

train_data = train_data.set_index('SMILES', drop=True)
train_data = shuffle(train_data, random_state=seed)
X_train = train_data.drop(columns=task)
y_train = train_data[task]

test_data = test_data.set_index('SMILES', drop=True)
X_test = test_data

current_model = SVR(kernel=kernel, C=C, epsilon=epsilon)
current = make_pipeline(MinMaxScaler(), current_model)
current.fit(X_train,y_train)
y_pred = current.predict(X_test)

In [18]:
y_pred

array([3.70941909, 2.79019503, 3.12436257, 2.36692634, 3.18326639,
       1.97317233, 1.98852173, 1.81229314, 1.37742727, 2.85706576,
       2.36917925, 1.93831704, 2.26421411, 2.54681267, 2.76730171,
       2.72416003, 2.42034072, 2.60674411, 2.48411796, 3.1756486 ,
       2.40167657, 1.45601041, 1.55012487, 2.95371899, 3.47302845,
       2.48005982, 2.85078172, 2.19825244, 1.81862765, 3.93411844,
       1.95085544, 2.89667553, 1.73166172, 2.85078172, 1.35772167,
       1.4520422 , 1.82275225, 1.2050014 , 0.74448151, 1.67999456,
       1.73051598, 1.69450758, 2.37379068, 1.23275773, 1.75059979,
       1.60031887, 3.34122561, 3.34122561, 1.55486428, 3.65280203,
       2.47641156, 2.62852615, 2.26723157, 2.29945925, 1.62414843,
       2.10079923, 1.46099683, 1.82613886, 2.13359313, 1.13539095,
       2.48702611, 2.6125398 , 2.77808608, 1.59524629, 2.8052702 ,
       1.96617633, 1.44134236, 1.29512872, 1.53137091, 1.32928438,
       2.25666722, 1.97065428, 1.50362846, 1.5993782 , 2.06931

In [19]:
df_admet_prediction['LogD'] = y_pred
df_admet_prediction

,HLM,KSOL,LogD
0,285.042364,235.351869,3.709419
1,346.422941,283.148616,2.790195
2,227.316115,240.038646,3.124363
3,248.095067,248.133280,2.366926
4,186.985961,376.532253,3.183266
...,...,...,...
121,103.581036,381.535459,1.958600
122,103.028499,371.569023,1.901163
123,75.458646,378.258895,2.109977
124,152.454394,424.290765,2.072440


MDR1

In [20]:
data = pd.read_csv('featurized_data_admet.csv').drop(columns=['my_id', 'KSOL', 'LogD', 'HLM', 'MLM'])
data

,SMILES,MDR1-MDCKII,training_set,0,1,2,3,4,5,6,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,2.0,True,1.171046,-1.709964,-1.232664,1.860159,-1.148111,-1.168615,1.584990,...,0.144337,1.531990,-0.810303,0.296689,-0.621365,-1.106144,0.762330,-0.761378,-1.274996,-0.460258
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,0.2,True,-2.745084,-1.049907,1.334981,1.905276,1.214691,0.937328,-2.540276,...,0.544986,1.725864,-0.310606,0.979101,0.459233,0.538236,-0.252584,-0.212149,-0.018674,-0.152451
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,0.5,True,-0.976000,-1.049323,3.051156,2.945424,2.015631,-1.640104,-0.486444,...,0.822131,1.796755,0.109316,1.148936,0.219072,0.434542,0.089569,0.211311,-0.209988,0.574256
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,8.5,True,-2.336900,-1.334151,-0.813121,5.999913,3.993001,-2.829812,-1.460422,...,0.330147,0.818578,-0.157749,0.460316,0.029226,1.010529,0.828361,0.412284,-1.456596,0.550043
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,0.9,True,-2.976158,0.377570,0.348468,2.663868,3.342389,-1.316121,-1.861355,...,0.974805,0.831901,-0.274851,1.695331,-0.544704,-0.302338,0.599298,-0.803252,-0.909561,0.617987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.600852,1.758608,3.050236,-2.634969,2.354528,-1.809476,-6.122465,...,0.676559,1.834212,-0.208483,1.160509,-0.379795,1.017874,0.023464,0.889000,-0.436904,1.477911
556,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.212416,1.016423,3.263524,-2.550871,2.011647,-2.047450,-6.484405,...,0.799475,1.806010,-0.320671,1.235600,-0.248747,1.048957,0.051964,0.893613,-0.487688,1.360138
557,COC[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN...,NaN,False,-0.356645,2.175031,3.133271,-2.253229,1.212674,-1.492542,-6.595284,...,1.134350,1.222514,-0.517434,1.147382,-0.472114,-0.133162,0.201345,0.679035,-0.680492,0.537756
558,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.364247,2.796600,3.689862,-2.036362,1.520496,-1.304389,-7.439854,...,0.973330,1.113834,-0.604300,0.977338,-0.599096,0.034397,0.263963,0.769886,-1.035524,1.216965


In [21]:
train_data = data[(data.training_set) & (~data['MDR1-MDCKII'].isna())].reset_index(drop=True).drop(columns=['training_set'])
test_data = data[~data.training_set].reset_index(drop=True).drop(columns=['training_set']).drop(columns='MDR1-MDCKII')

In [22]:
task = 'MDR1-MDCKII'
kernel = 'poly'
C = 0.04
epsilon = 0.30
seed = 1

train_data = train_data.set_index('SMILES', drop=True)
train_data = shuffle(train_data, random_state=seed)
X_train = train_data.drop(columns=task)
y_train = train_data[task]

test_data = test_data.set_index('SMILES', drop=True)
X_test = test_data

current_model = SVR(kernel=kernel, C=C, epsilon=epsilon)
current = make_pipeline(MinMaxScaler(), current_model)
current.fit(X_train,y_train)
y_pred = current.predict(X_test)

In [23]:
y_pred

array([14.22324959, 16.8573196 , 13.45923725, 15.87702895, 17.25552767,
       10.98520867, 10.37845762, 19.08786662, 17.17359443,  0.59898852,
        0.940637  , 16.40167131,  1.44864904, 17.62711134,  1.01921964,
       19.29157406, 16.91118485, 17.39053527,  1.60158335, 19.48534467,
        1.60951012, 18.0129626 , 17.46253185, 17.05115146,  5.98790811,
        8.99835829, 14.27228579, 18.15755082, 19.4666785 ,  6.55163998,
       11.95578964, 14.04752811, 11.26662426, 14.27228579, 17.80228871,
       18.83314884, 17.34629706, 18.07391044, 16.05996483, 19.95059061,
       19.07202597, 18.19905865, 17.3526239 , 10.77790617, 11.62516404,
       18.54881561, 14.71094692, 14.71094692, 17.60272668,  8.28886937,
        8.9986148 , 12.79430082, 19.16302455, 17.75287358, 17.35225868,
       19.90010927,  9.27175641, 19.55206141, 20.26599519, 16.66336891,
       10.04208331, 20.92741758, 18.85694578, 18.33803738, 18.74642359,
        1.71994452,  0.4796929 ,  0.39312542,  0.79486504,  1.34

In [24]:
df_admet_prediction['MDR1-MDCKII'] = y_pred
df_admet_prediction

,HLM,KSOL,LogD,MDR1-MDCKII
0,285.042364,235.351869,3.709419,14.223250
1,346.422941,283.148616,2.790195,16.857320
2,227.316115,240.038646,3.124363,13.459237
3,248.095067,248.133280,2.366926,15.877029
4,186.985961,376.532253,3.183266,17.255528
...,...,...,...,...
121,103.581036,381.535459,1.958600,5.444568
122,103.028499,371.569023,1.901163,5.442214
123,75.458646,378.258895,2.109977,3.434749
124,152.454394,424.290765,2.072440,4.153509


MLM

In [25]:
data = pd.read_csv('featurized_data_admet.csv').drop(columns=['my_id', 'KSOL', 'MDR1-MDCKII', 'HLM', 'LogD'])
data

,SMILES,MLM,training_set,0,1,2,3,4,5,6,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,NaN,True,1.171046,-1.709964,-1.232664,1.860159,-1.148111,-1.168615,1.584990,...,0.144337,1.531990,-0.810303,0.296689,-0.621365,-1.106144,0.762330,-0.761378,-1.274996,-0.460258
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,NaN,True,-2.745084,-1.049907,1.334981,1.905276,1.214691,0.937328,-2.540276,...,0.544986,1.725864,-0.310606,0.979101,0.459233,0.538236,-0.252584,-0.212149,-0.018674,-0.152451
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,NaN,True,-0.976000,-1.049323,3.051156,2.945424,2.015631,-1.640104,-0.486444,...,0.822131,1.796755,0.109316,1.148936,0.219072,0.434542,0.089569,0.211311,-0.209988,0.574256
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,NaN,True,-2.336900,-1.334151,-0.813121,5.999913,3.993001,-2.829812,-1.460422,...,0.330147,0.818578,-0.157749,0.460316,0.029226,1.010529,0.828361,0.412284,-1.456596,0.550043
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,NaN,True,-2.976158,0.377570,0.348468,2.663868,3.342389,-1.316121,-1.861355,...,0.974805,0.831901,-0.274851,1.695331,-0.544704,-0.302338,0.599298,-0.803252,-0.909561,0.617987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.600852,1.758608,3.050236,-2.634969,2.354528,-1.809476,-6.122465,...,0.676559,1.834212,-0.208483,1.160509,-0.379795,1.017874,0.023464,0.889000,-0.436904,1.477911
556,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.212416,1.016423,3.263524,-2.550871,2.011647,-2.047450,-6.484405,...,0.799475,1.806010,-0.320671,1.235600,-0.248747,1.048957,0.051964,0.893613,-0.487688,1.360138
557,COC[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN...,NaN,False,-0.356645,2.175031,3.133271,-2.253229,1.212674,-1.492542,-6.595284,...,1.134350,1.222514,-0.517434,1.147382,-0.472114,-0.133162,0.201345,0.679035,-0.680492,0.537756
558,C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(C...,NaN,False,0.364247,2.796600,3.689862,-2.036362,1.520496,-1.304389,-7.439854,...,0.973330,1.113834,-0.604300,0.977338,-0.599096,0.034397,0.263963,0.769886,-1.035524,1.216965


In [26]:
train_data = data[(data.training_set) & (~data['MLM'].isna())].reset_index(drop=True).drop(columns=['training_set'])
test_data = data[~data.training_set].reset_index(drop=True).drop(columns=['training_set']).drop(columns='MLM')

In [27]:
task = 'MLM'
kernel = 'poly'
C = 7.25
epsilon = 0.45
seed = 1

train_data = train_data.set_index('SMILES', drop=True)
train_data = shuffle(train_data, random_state=seed)
X_train = train_data.drop(columns=task)
y_train = train_data[task]

test_data = test_data.set_index('SMILES', drop=True)
X_test = test_data

current_model = SVR(kernel=kernel, C=C, epsilon=epsilon)
current = make_pipeline(MinMaxScaler(), current_model)
current.fit(X_train,y_train)
y_pred = current.predict(X_test)

In [28]:
y_pred

array([ 544.04841676,  323.6506006 ,  410.03572006,  347.77873088,
        369.65983605,  380.73674841,  482.96296016,  353.02825194,
        470.71195003,  182.3873115 ,  159.55764893,  309.39959527,
        184.76360345,  340.49502854,  195.56471484,  451.21881835,
        327.34563035,  342.25001008,  210.23396008,  505.91673834,
        195.9572419 ,  313.30471541,  313.1252557 ,  304.22196591,
       1095.48035366, 1066.38765056,  511.48138701,  300.2668694 ,
        392.36098954, 1160.41033726,  466.12331877,  521.23575171,
        406.47313862,  511.48138701,  321.88161028,  475.06982677,
        313.5010882 ,  500.82514012,  357.71575361,  542.94270984,
        545.29727321,  351.85582615,  348.21646835,  300.58465944,
        327.47154477,  265.82296355,  536.73311796,  536.73311796,
        506.94917644, 1191.09776158, 1084.79876881,  525.39368518,
        357.47561913,  378.85134654,  476.9511359 ,  468.09275457,
        283.26851476,  537.28809682,  404.70628017,  441.90749

In [29]:
df_admet_prediction['MLM'] = y_pred
df_admet_prediction

,HLM,KSOL,LogD,MDR1-MDCKII,MLM
0,285.042364,235.351869,3.709419,14.223250,544.048417
1,346.422941,283.148616,2.790195,16.857320,323.650601
2,227.316115,240.038646,3.124363,13.459237,410.035720
3,248.095067,248.133280,2.366926,15.877029,347.778731
4,186.985961,376.532253,3.183266,17.255528,369.659836
...,...,...,...,...,...
121,103.581036,381.535459,1.958600,5.444568,273.371881
122,103.028499,371.569023,1.901163,5.442214,279.305363
123,75.458646,378.258895,2.109977,3.434749,227.051443
124,152.454394,424.290765,2.072440,4.153509,240.352616


In [ ]:
df_admet_prediction.to_csv('final_admet_predictions.csv', index=False)